<a href="https://colab.research.google.com/github/entc-17-fyp-20/CM_Data_Visualizing/blob/samal/outlier_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load the dataset

In [2]:
pip install pycaret

     |████████████████████████████████| 264 kB 5.3 MB/s 
     |████████████████████████████████| 56 kB 3.0 MB/s 
     |████████████████████████████████| 6.8 MB 46.5 MB/s 
     |████████████████████████████████| 248 kB 29.7 MB/s 
     |████████████████████████████████| 80 kB 7.4 MB/s 
     |████████████████████████████████| 271 kB 35.3 MB/s 
     |████████████████████████████████| 111 kB 46.0 MB/s 
     |████████████████████████████████| 167 kB 62.5 MB/s 
     |████████████████████████████████| 14.6 MB 81 kB/s 
     |████████████████████████████████| 2.0 MB 62.0 MB/s 
     |████████████████████████████████| 1.7 MB 52.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 32.0 MB/s 
     |████████████████████████████████| 3.1 MB 59.8 MB/s 
     |████████████████████████████████| 675 kB 50.6 MB/s 
     |███████████████████

### Dimension reduction upto 10 variables

Original dataset contains 23 features. Following features has a correlation coefficient greater than 0.8 ( highly correlated) 

* Voltage Phase R                  
* Voltage Phase S                   
* Voltage Phase T  


---


* Current Phase R                    
* Current Phase S                  
* Current Phase T 


---


* Gearbox oil Temperature            
* Gearbox Bearing Temperature       
* Generator Winding Temperature R    
* Generator Winding Temperature S    
* Generator Winding Temperature T 
* Main bearing Temperature         
* Generator Bearing Temperature DE   
* Generator Bearing Temperature NDE  

---

Those features averaged to form only 3 features



In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

url1 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/S6%2010%20seconed%20data.csv' #S6 10 seconed data.csv
url2 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V08%20June%2010%20sec%20data.csv?token=ANECSWYTSJHLNER56T5JDR3BCLBDK' #V08 June 10 sec data.csv
url3 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V13%20June%2010%20sec%20data.csv?token=ANECSW37NCJ7TMWXNPKFHM3BCLBK2' #V13 June 10 sec data.csv
df5 = pd.read_csv(url3)


df5["Gearbox & Bearing temp"]  = (df5["Gearbox Bearing Temperature"] + df5["Gearbox oil Temperature"]+ df5["Generator Winding Temperature R"]+df5["Generator Winding Temperature S"]+df5["Generator Winding Temperature T"]+df5["Main bearing Temperature"] + df5["Generator Bearing Temperature DE"]+ df5["Generator Bearing Temperature NDE"])/8


df5.drop(["Reactive Power kVAR", "Frequency Hz","Voltage Phase R","Voltage Phase S","Voltage Phase T","Current Phase R","Current Phase S","Current Phase T","Gearbox Bearing Temperature","Gearbox oil Temperature","Generator Winding Temperature R","Generator Winding Temperature S","Generator Winding Temperature T","Main bearing Temperature","Generator Bearing Temperature DE","Generator Bearing Temperature NDE",'Hydraulic unit Pressure in bar'], axis='columns', inplace=True)
df5_copy = df5.copy()
df5.describe()

df5['Date & Time'] = pd.to_datetime(df5['Date & Time'], format="%d/%m/%Y %H:%M:%S")
df5.set_index('Date & Time', inplace = True)
df5= df5.resample('10min').mean().reset_index()
df5=df5.head(3100)
df5_new = df5[['Date & Time']]


### Outlier detection using iforest model and replcement by moving average

In [54]:
column_list = [ 'Nacelle Temperature', 'Ambient Temperature','Hydraulic oil Temperature','Gearbox & Bearing temp']
for column_name in column_list:


  df5_sub = df5[['Date & Time',column_name]]
  df5_sub['MA48'] = df5_sub[column_name].rolling(16,center=True,min_periods=1).mean()

  from pycaret.anomaly import *
  s = setup(df5_sub, session_id = 123)

  iforest = create_model('iforest', fraction = 0.01)
  iforest_results = assign_model(iforest)

  df5_new[column_name+' new'] = iforest_results['MA48'].values*iforest_results['Anomaly'].values+iforest_results[column_name].values*(1-iforest_results['Anomaly'].values)

In [55]:
for column_name in column_list:
  fig = px.line(df5, x="Date & Time" , y=column_name) 
  fig.show()

In [56]:
from scipy.interpolate import interp1d

x = np.linspace(-5, 28, num=34, endpoint=True)
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 75, 150, 240, 362.5, 512.5, 670, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800])
f = interp1d(x, y, kind='cubic') # creating function


In [57]:
error_margin = 0.45 #@param {type:"slider", min:0.1, max:0.6, step:0.05}

def fil(x):
  return int(f(x))

def compare(x):
  actual = x["Active Power kW"]
  calculated = x["Power_cal"]

  if calculated == 0:
    error = abs(actual-calculated)/0.001
    if error>error_margin:
      return calculated
    return actual

  error = abs(actual-calculated)/calculated
  if error>error_margin:
    return calculated
  return actual

def addcol(x):
  actual = x["Active Power kW"]
  calculated = x["Power_cal"]

  if calculated == 0:
    error = abs(actual-calculated)/0.001
    if error>error_margin:
      return "replaced"
    return "actual"

  error = abs(actual-calculated)/calculated
  if error>error_margin:
    return "replaced"
  return "actual"

In [58]:
df5["Power_cal"] = df5["Wind Speed m/s"].apply(fil) # calculating power from wind speed
df5["Active Power"] = df5.apply(compare, axis=1) # Replacing outliers with calculated power
df5["Power_cate"] = df5.apply(addcol, axis=1) # Addting outlier/not outlier column

In [59]:
df5.drop(['Active Power kW','Generator RPM','Power_cal','Power_cate'], axis='columns', inplace=True)

In [60]:
df5

,Date & Time,Wind Speed m/s,Nacelle Temperature,Ambient Temperature,Hydraulic oil Temperature,Gearbox & Bearing temp,Active Power
0,2021-06-02 00:00:00,8.900000,31.150000,30.746667,45.848333,83.150208,475.936667
1,2021-06-02 00:10:00,8.708333,31.220000,30.790000,45.911667,82.710208,447.813333
2,2021-06-02 00:20:00,8.853333,30.775000,30.683333,45.861667,83.413958,485.601667
3,2021-06-02 00:30:00,8.731667,30.446667,30.356667,45.968333,82.530833,459.966667
4,2021-06-02 00:40:00,9.021667,30.206667,30.328333,45.965000,82.909792,497.221667
...,...,...,...,...,...,...,...
3095,2021-06-23 11:50:00,10.346667,30.478333,31.576667,47.935000,92.653333,726.665000
3096,2021-06-23 12:00:00,10.041667,30.590000,31.703333,47.836667,92.449792,696.938333
3097,2021-06-23 12:10:00,10.790000,30.411667,31.491667,47.951667,93.257292,739.048333
3098,2021-06-23 12:20:00,10.756667,30.741667,31.663333,47.920000,93.771875,742.513333
